# DT Assignment1

# Data Loading

In [25]:
import pandas as pd 
import numpy as np

In [26]:
pd_data = pd.read_csv('https://raw.githubusercontent.com/AugustLONG/ML01/master/01decisiontree/AllElectronics.csv')
pd_data.drop("RID",axis=1, inplace = True) #RID는 그냥 순서라서 삭제
pd_data

,age,income,student,credit_rating,class_buys_computer
0,youth,high,no,fair,no
1,youth,high,no,excellent,no
2,middle_aged,high,no,fair,yes
3,senior,medium,no,fair,yes
4,senior,low,yes,fair,yes
5,senior,low,yes,excellent,no
6,middle_aged,low,yes,excellent,yes
7,youth,medium,no,fair,no
8,youth,low,yes,fair,yes
9,senior,medium,yes,fair,yes


# 1. Gini 계수를 구하는 함수 만들기

- Input: df(데이터), label(타겟변수명)
- 해당 결과는 아래와 같이 나와야 합니다.

In [96]:
# gini index: 변수의 값이 yes일때와 no일 때의 확률을 제곱의 차를 통해 구할 수 있다
def get_gini(df, label):
    if len(df)==0:
        gini = 1
    else: 
        yes = sum(df[label]=='yes') / len(df)
        no = sum(df[label]=='no') / len(df)
        gini = 1 - (yes)**2 - (no)**2
    
    return gini    

In [97]:
get_gini(pd_data,'class_buys_computer')

0.4591836734693877

# 2. Feature의 Class를 이진 분류로 만들기
 ## ex) {A,B,C} -> ({A}, {B,C}), ({B}, {A,C}), ({C}, {A,B})

- Input: df(데이터), attribute(Gini index를 구하고자 하는 변수명)
- 해당 결과는 아래와 같이 나와야 합니다.

In [98]:
import itertools 
def get_binary_split(df, attribute):
    unique = list(df[attribute].unique()) #unique 한 값들 list에 저장
    double = list(itertools.combinations(unique,2)) #itertools 사용해 2개의 combinations들 double list에 저장
    
    result = [] #결과 저장할 list
    for uni in unique:
        result.append([uni])
    for i in range(len(double)):
        result.append(list(double[i])) # 값 하나마다 list 추가

    return result

In [99]:
get_binary_split(pd_data,"age")

[['youth'],
 ['middle_aged'],
 ['senior'],
 ['youth', 'middle_aged'],
 ['youth', 'senior'],
 ['middle_aged', 'senior']]

# 3. 다음은 모든 이진분류의 경우의 Gini index를 구하는 함수 만들기
- 위에서 완성한 두 함수를 사용하여 만들어주세요!
- 해당 결과는 아래와 같이 나와야 합니다.

In [100]:
def get_attribute_gini_index(df, attribute, label):
        #binary_split() 함수로부터 split한 변수들 list로 받기
        binary_split = get_binary_split(df, attribute) 
        result = {} #결과값의 형태에 맞추어 result 저장할 dictionary 생성
        true_df = pd.DataFrame()
        false_df = pd.DataFrame()

        #모든 이진분류에 대해 gini index 
        for i in binary_split:
                true = df[attribute].isin(i)#attribute와  binary split 한 것과 일치할 때
                true_df = df[true] #dataframe에 저장
                false = ~df[attribute].isin(i)#attribute와 binary split 한 값과 일치하지 않을 때
                false_df = df[false]

                #get_gini()함수를 통해 이진분류의 gini index 구하기
                gini = (len(true_df)/len(df))*get_gini(true_df, label) + (len(false_df)/len(df))*get_gini(false_df, label) 

                #tuple 형태의 key 값 str.join() 활용해 string으로 변환
                #각 binary_split에 해당하는 gini index result dictionary에 저장
                result[''.join(tuple(i))] = gini

        return result

In [101]:
get_attribute_gini_index(pd_data, "age", "class_buys_computer")

{'youth': 0.3936507936507937,
 'middle_aged': 0.35714285714285715,
 'senior': 0.4571428571428572,
 'youthmiddle_aged': 0.4571428571428572,
 'youthsenior': 0.35714285714285715,
 'middle_agedsenior': 0.3936507936507937}

여기서 가장 작은 Gini index값을 가지는 class를 기준으로 split해야겠죠?

In [102]:
min(get_attribute_gini_index(pd_data, "age", "class_buys_computer").items())

('middle_aged', 0.35714285714285715)

# 다음의 문제를 위에서 작성한 함수를 통해 구한 값으로 보여주세요!
## 문제1) 변수 ‘income’의 이진분류 결과를 보여주세요.

## 문제2) 분류를 하는 데 가장 중요한 변수를 선정하고, 해당 변수의 Gini index를 제시해주세요.

## 문제3) 문제 2에서 제시한 feature로 DataFrame을 split한 후 나눠진 2개의 DataFrame에서 각각   다음으로 중요한 변수를 선정하고 해당 변수의 Gini index를 제시해주세요.

In [103]:
##문제1 답안
get_binary_split(pd_data, 'income')

[['high'],
 ['medium'],
 ['low'],
 ['high', 'medium'],
 ['high', 'low'],
 ['medium', 'low']]

In [108]:
##문제2 답안
# 분류를 하는데 있어 가장 중요한 변수 -> 모든 변수들의 gini index 확인한 것의 min 값들 비교 
feature = list(pd_data.columns)
del feature[-1]
feature

['age', 'income', 'student', 'credit_rating']

In [110]:
#각 feature 별로 minimum gini index 찾아서 비교
for i in feature:
    print(i, min(get_attribute_gini_index(pd_data, i, "class_buys_computer").items()))

#4개의 feature 중 'age'의 minimum gini index가 가장 작으므로 가장 중요한 feature는 'age'이다
#age의 gini index: 'middle_aged', 0.35714285714285715

age ('middle_aged', 0.35714285714285715)
income ('high', 0.4428571428571429)
student ('no', 0.3673469387755103)
credit_rating ('excellent', 0.42857142857142855)


In [91]:
##문제3 답안
# age 변수 중 middle aged 가 minimum gini index를 가져 주요 변수로 선정

